In [1]:
from hourly.hourly import get_work_commits, get_labor, plot_labor

In [2]:
help(plot_labor)

Help on function plot_labor in module hourly.hourly:

plot_labor(labor, freq, name=None)



In [3]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go

import pandas as pd

In [4]:
init_notebook_mode(connected = True)

In [5]:
import json
import os

In [6]:
def get_hourly_config(gitdir):
    """Loads hourly config file from .hourly_config.json"""
    hourly_config_file = gitdir + '/.hourly-config.json'
    if os.path.exists(hourly_config_file):
        with open(hourly_config_file) as f:
            hourly_config = json.load(f)
            return hourly_config
#     else:
#         print('no hourly_config.json found in {}'.format(gitdir))

def get_errant_clocks(config):
    errant_clocks = []
    if config is not None:
        errant_clocks = config.get('errant_clocks',[])
    return errant_clocks



In [31]:
gitdirs = ['/Users/apembrok/git/hourly',
           '/Users/apembrok/git/PYSAT',
          '/Users/apembrok/git/kameleon',
          '/Users/apembrok/git/kamodo/',
          '/Users/apembrok/git/mms',
#           '/Users/apembrok/git/mms_pysat',
          ]

In [37]:
tdelta = labor.set_index('TimeIn').TimeDelta.groupby(pd.Grouper(freq = '7d')).sum()


In [40]:
tdelta.asfreq('7d')

TimeIn
2019-08-03 00:00:00-04:00   0 days 04:04:53
2019-08-10 00:00:00-04:00   0 days 00:31:41
2019-08-17 00:00:00-04:00   0 days 21:52:07
2019-08-24 00:00:00-04:00   1 days 03:18:10
2019-08-31 00:00:00-04:00   0 days 07:40:01
2019-09-07 00:00:00-04:00   0 days 03:56:00
2019-09-14 00:00:00-04:00   0 days 00:00:00
2019-09-21 00:00:00-04:00   0 days 05:07:24
2019-09-28 00:00:00-04:00   0 days 00:00:00
2019-10-05 00:00:00-04:00   0 days 00:00:00
2019-10-12 00:00:00-04:00   0 days 00:00:00
2019-10-19 00:00:00-04:00   0 days 03:45:59
Freq: 7D, Name: TimeDelta, dtype: timedelta64[ns]

In [41]:
def plot_labor(labor, freq, name = None):
    tdelta = labor.set_index('TimeIn').TimeDelta.groupby(pd.Grouper(freq = freq)).sum().asfreq(freq)

    tdelta_trace = go.Bar(x = pd.Series(tdelta.index), 
                          y = [td.total_seconds()/3600 for td in tdelta],
                          text = [str(td) for td in tdelta],
                          name = name)
    return tdelta_trace

In [49]:
traces = []
for gitdir in gitdirs:
    name = gitdir.strip('/').split('/')[-1]
    work, repo = get_work_commits(gitdir, ascending = True, tz = 'US/Eastern')
    errant_clocks = get_errant_clocks(get_hourly_config(gitdir))
    labor = get_labor(work, errant_clocks= errant_clocks, verbose= False, match_logs = False)
    labor_trace = plot_labor(labor, '1d', name)
    traces.append(labor_trace)
    

iplot(go.Figure(traces, layout = dict(barmode = 'stack')))